In [1]:
import mcl
mcl.approx_pi(1000)

3.1406380562059946

In [21]:
import pyzx as zx
import json

# Allows us to visualize circuits from qasm
from cirq.contrib.qasm_import import circuit_from_qasm


ocaml_output = """
[
  { "Prep": { "on_qubits": [ 0 ] } },
  { "Prep": { "on_qubits": [ 1 ] } },
  { "Prep": { "on_qubits": [ 2 ] } },
  { "XCorrect": { "on_qubits": [ 1 ], "signals": [] } }
]
"""

#{ "Entangle": { "on_qubits": [ 0, 1 ] } }
measure_string = """
  {
    "Measure": {
      "on_qubits": [ 0 ],
      "angle": 0.0,
      "signal_s": [],
      "signal_t": []
    }
  }
"""

loaded_program = json.loads(ocaml_output)

from mcl import ZXBuilder
zx_graph = ZXBuilder(loaded_program)
qasm = zx_graph.to_qasm()
zx.draw(zx_graph.to_circuit())
zx.draw(zx_graph.graph)
print(qasm)
circuit = circuit_from_qasm(qasm)

print(circuit)

print('\n')

from mcl import CirqBuilder
circuit = CirqBuilder(loaded_program)
qasm = circuit.to_qasm()
print(qasm)
print(circuit.to_circuit())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
h q[2];
h q[1];
h q[0];
h q[1];
rz(1.0*pi) q[1];

q_0: ───H───────────────

q_1: ───H───H───Rz(π)───

q_2: ───H───────────────


// Generated from Cirq v0.12.0

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [0, 1, 2]
qreg q[3];


h q[0];
h q[1];
h q[2];
x q[1];

0: ───H───────

1: ───H───X───

2: ───H───────
